In [ ]:
!sudo apt-get install zip -y

In [27]:
!mkdir lambda_layer
!mkdir lambda_layer/python

In [45]:
!pip install sagemaker==2.224.1 -t lambda_layer/python/ --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 237.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 198.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 191.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 284.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 215.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 162.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 320.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 348.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 198.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 249.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 265.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [42]:
!find lambda_layer/python/ -type d -name "__pycache__" -exec rm -r {} +
!find lambda_layer/python/ -type d -name "tests" -exec rm -r {} +
!rm -r lambda_layer/python/**/*.dist-info

rm: cannot remove 'lambda_layer/python/**/*.dist-info': No such file or directory


In [46]:
!zip -r9 -q lambda_layer.zip lambda_layer/

In [48]:
!aws lambda publish-layer-version \
    --layer-name sagemaker-layer \
    --description "Lambda layer with sagemaker library" \
    --zip-file fileb://lambda_layer.zip \
    --compatible-runtimes python3.8 python3.9 python3.10 python3.11


An error occurred (RequestEntityTooLargeException) when calling the PublishLayerVersion operation: Request must be smaller than 70167211 bytes for the PublishLayerVersion operation


In [20]:
!pip install -r requirements.txt -t .

  Using cached sagemaker-2.224.1-py3-none-any.whl.metadata (15 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached boto3-1.34.143-py3-none-any.whl.metadata (6.6 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl.metadata (943 bytes)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached pathos-0.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached schema-0.7.7-py2.py3-none-any.whl.metadata (34 kB)
  Using cached PyY

In [21]:
import boto3

def delete_sagemaker_pipeline(pipeline_name):
    # Create a SageMaker client
    sagemaker_client = boto3.client('sagemaker')

    try:
        # Delete the pipeline
        response = sagemaker_client.delete_pipeline(PipelineName=pipeline_name)
        print(f"Pipeline '{pipeline_name}' has been deleted successfully.")
    except sagemaker_client.exceptions.ResourceNotFound as e:
        print(f"Pipeline '{pipeline_name}' does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    pipeline_name = '06-full-pipeline'  # Replace with your SageMaker pipeline name
    delete_sagemaker_pipeline(pipeline_name)


Pipeline '06-full-pipeline' does not exist.


In [22]:
import os
import zipfile

def zip_lambda_function():
    zip_filename = 'function.zip'
    
    # Create a Zip file
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as lambda_zip:
        # Walk through the directory and add files to the zip file
        for root, dirs, files in os.walk('.'):
            for file in files:
                lambda_zip.write(os.path.join(root, file))
    
    print(f'{zip_filename} created successfully.')

if __name__ == "__main__":
    zip_lambda_function()

function.zip created successfully.


In [23]:
!aws lambda create-function \
    --function-name create-sagemaker-pipeline \
    --zip-file fileb://function.zip \
    --handler lambda_function.lambda_handler \
    --runtime python3.8 \
    --role arn:aws:iam::211125740051:role/lambda_pipeline


An error occurred (RequestEntityTooLargeException) when calling the CreateFunction operation: Request must be smaller than 70167211 bytes for the CreateFunction operation


In [47]:
import os

def get_file_size(file_path):
    try:
        # Get the size of the file in bytes
        file_size_bytes = os.path.getsize(file_path)
        
        # Convert the size to kilobytes and megabytes
        file_size_kb = file_size_bytes / 1024
        file_size_mb = file_size_bytes / (1024 * 1024)
        
        return file_size_bytes, file_size_kb, file_size_mb
    except OSError as e:
        print(f"Error: {e}")
        return None, None, None

# Path to the file
file_path = 'lambda_layer.zip'

# Get the size of the file
file_size_bytes, file_size_kb, file_size_mb = get_file_size(file_path)

# Print the size of the file in different units
if file_size_bytes is not None:
    print(f"File size of '{file_path}':")
    print(f"{file_size_bytes:.2f} bytes")
    print(f"{file_size_kb:.2f} KB")
    print(f"{file_size_mb:.2f} MB")
else:
    print("Failed to get the file size.")


File size of 'lambda_layer.zip':
65242454.00 bytes
63713.33 KB
62.22 MB
